# JDA Test Task  
### Камалетдинова Юлия  
### anacriel@protonmail.com
--------------------------

## Задание 1

#### 1. Запрос для получения среднего по актуальным ценам

Напишите запрос, возвращающий строки с актуальными ценами (с наибольшей датой - `date_start`) для каждого товара (`product`).  
Посчитайте среднюю цену и запишите ответ - число в файл `res1.txt`

``` sql
SELECT avg(a.price)  
FROM prices a  
INNER JOIN (  
    SELECT price, MAX(date_start) date_start  
    FROM prices  
    GROUP BY product  
) b ON a.price = b.price and a.date_start = b.date_start;
```

#### 2. Стоимости и соответствующие даты для продукта `product`

Напишите функцию на любом языке программирования (предпочтительно **Python**) принимающую название продукта  
(поле `product`) и возвращающую tuple вида (`[date_start1, date_start2,... ],[price1, price2,...]`) из таблицы  
`prices`, где `date_start(n)` –– это даты, отсортированные по возрастанию, а `price(n)` - соответствующая  дате цена на продукт.  
При каждом вызове функция должна обращаться к БД, выбирая необходимые данные. Формат дат особого значения не имеет

In [1]:
import os
import sqlite3

from contextlib import closing
from typing import Tuple, List

In [2]:
DB_FILE = 'DB.db'
DEFAULT_PATH = os.path.join(os.getcwd(), DB_FILE)

In [3]:
def db_connect(db_path: str = DEFAULT_PATH) -> sqlite3.Connection:
    """
    Create and return connection to DB.
    
    Args:
        db_path: path to the database file
    Returns:
        sqlite3.Connection to the database
    """

    conn = sqlite3.connect(db_path)
    return conn

In [4]:
def get_product_data(product: str) -> Tuple[List[str], List[float]]:
    """
    Connect to DB, get price history for the 
    product from `prices` table.

    Args:
        product: the product name.
    Returns:
        tuple of dates and prices corresponding to
        product, ordered by ascending date:
        ([date_start1, date_start2, ...], [price1, price2, ...])
    """

    conn = db_connect()
    with closing(conn.cursor()) as cur:
        cur.execute(
            "SELECT date_start, price FROM prices WHERE product = ?"
            "ORDER BY date_start ASC",
            (product,),
        )

        rows = cur.fetchall()
        dates_prices = tuple(map(list, zip(*rows)))

        return dates_prices

In [5]:
# Example usage
get_product_data('A')

(['1970-01-01',
  '2010-01-21',
  '2010-08-17',
  '2010-11-02',
  '2012-08-27',
  '2012-12-06',
  '2013-01-11',
  '2013-05-02',
  '2013-05-08',
  '2013-11-01',
  '2014-08-30',
  '2015-12-29',
  '2016-05-04',
  '2016-08-14',
  '2016-09-28',
  '2017-04-01'],
 [297.79,
  164.68,
  558.58,
  127.08,
  352.13,
  0.35,
  100.61,
  120.5,
  260.19,
  1747.84,
  59.84,
  118.27,
  550.62,
  339.35,
  210.19,
  111.23])

## Задание 2

#### 1. Запрос для создания таблицы `revenue`

Напишите запрос, результатом которого станет создание таблицы `revenue`, где `revenue` вычисляется по формуле: 
**revenue = sales.amount * prices.price**. Естественно, `price` из таблицы `prices` должен соответствовать `date` из таблицы `sales`.  
Следует учитывать, что таблица `sales` содержит большое кол-во строк.
Выполните запрос и создайте таблицу в revenue `"DB.db"`.  

Запишите величину выборочного стандартного отклонения `revenue` для продукта `"B"` в файл `res2.txt`.

``` sql
CREATE TABLE revenue
    AS SELECT s.*,
            (SELECT p.price
             FROM prices p
             WHERE p.date_start <= s.date AND p.product = s.product
             ORDER BY p.date_start DESC
             LIMIT 1
            ) * s.amount AS revenue
        FROM sales s;
```

#### 2. Функция для расчета выборочного стандартного отклонения `revenue`

In [6]:
import numpy as np

In [7]:
def get_ssd(prod_name: str) -> np.array:
    """
    Connect to DB, request revenue by product 
    and count its sample standard deviation.

    Args:
        prod_name: the product name.
    Returns:
        sample standard deviation
    """
    
    conn = db_connect()
    with closing(conn.cursor()) as cur:
        cur.execute("SELECT revenue FROM revenue WHERE product = ?;", (prod_name,))

        rows = cur.fetchall()
        rev = np.asarray(list(zip(*rows))[0])

        ssd = np.std(rev, ddof=1)

        return ssd

In [8]:
# Example usage
get_ssd('B')

33465.93838814805

## Задание 3

Напишите функцию, которая принимает название товара `product` и две даты: `date_start`,
`date_end`, тренирует  
(находит c МНК коэффициенты **k**, **b**) линейную модель вида **y = kx+b** и возвращает коэффициенты, **k** и **b**. 
В модели **x** - это номер дня  
от начала заданного периода (0, 1, 2, 3...), а **y** - кол-во проданных единиц продукции (`amount` из таблицы `sales` или `revenue`).  
При каждом вызове функция должна обращаться к таблице `revenue` или `sales` из **Задания 2**, выбирая необходимые данные.  

Запишите коэффициенты, возвращаемые функцией при начальных аргументах: `lin_reg(product='Q', date_start='2019-01-01', date_end='2019-03-31')` в `res3.txt` через пробел.


In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
def lin_reg(product: str, date_start: str, date_end: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Connect to DB, request periods and amounts sold,
    train Least Squares Regression model y = kx + b and
    return k, b coefficients

    Args:
        product: the product name.
        date_start: starting date in `YYYY-MM-DD` format
        date_end: ending date in `YYYY-MM-DD` format
    Returns:
        coefficients k, b of the lm model
    """
    conn = db_connect()
    with closing(conn.cursor()) as cur:
        # Select amounts and date periods corresponding
        # to the product
        cur.execute(
            "SELECT julianday(date) - julianday(?),"
            "amount FROM revenue WHERE date > ? AND "
            "date < ? AND product = ?;",
            (date_start, date_start, date_end, product,),
        )
        rows = cur.fetchall()

        # Transform query data
        days_amounts = list(zip(*rows))
        x = np.array([int(day) for day in days_amounts[0]]).reshape(-1, 1)
        y = np.array(days_amounts[1]).reshape(-1, 1)

        # Train linear model
        model = LinearRegression()
        model.fit(x, y)
        
        return model.coef_[0], model.intercept_

In [11]:
lin_reg(product='Q', date_start='2019-01-01', date_end='2019-03-31')

(array([-0.16513156]), array([28.95062696]))